<a href="https://colab.research.google.com/github/longevity-genie/just-agents/blob/main/examples/notebooks/02_sqlite_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Building your Own SQL Agent with Just-Agents ##


This notebook provides a comprehensive guide on how to create and utilize your own custom SQL agents using [just-agents](https://github.com/longevity-genie/just-agents/) library.

It is the second tutorial in a series of tutorials, next ones are:
1. Basic agents tutorial  
2. Database agent tutorial (THIS ONE) 
3. Coding agent tutorial 

## Working with the Open-Genes Database

For this tutorial, we'll be interacting with the Open-Genes database. We'll be using a simplified version stored in an SQLite file. You can access the original database file at:

[https://github.com/longevity-genie/longevity_gpts/blob/main/open_genes/data/open_genes.sqlite](https://github.com/longevity-genie/longevity_gpts/blob/main/open_genes/data/open_genes.sqlite)

Let's download the database first

In [11]:
!wget https://github.com/longevity-genie/longevity_gpts/raw/main/open_genes/data/open_genes.sqlite -O open_genes.sqlite

--2025-03-01 20:22:11--  https://github.com/longevity-genie/longevity_gpts/raw/main/open_genes/data/open_genes.sqlite
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/longevity-genie/longevity_gpts/main/open_genes/data/open_genes.sqlite [following]
--2025-03-01 20:22:12--  https://raw.githubusercontent.com/longevity-genie/longevity_gpts/main/open_genes/data/open_genes.sqlite
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8003::154, 2606:50c0:8000::154, 2606:50c0:8001::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8003::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1150976 (1.1M) [application/octet-stream]
Saving to: ‘open_genes.sqlite’

open_genes.sqlite   100%[===================>]   1.10M  --.-KB/s    in 0.1s    

2025-0

Let's test the query by outputing the tables

In [12]:
import sqlite3

def db_query(sql: str):
    """ This function execute query on open-genes sqlite table. It returns query results. """
    conn = sqlite3.connect("open_genes.sqlite")
    cursor = conn.cursor()

    cursor.execute(sql)
    try:
        rows = cursor.fetchall()
        if rows is None or len(rows) == 0:
            conn.close()
            return ""
        names = [description[0] for description in cursor.description]
        text = "; ".join(names) + "\n"
        for row in rows:
            row = [str(i) for i in row]
            text += "; ".join(row) + "\n"
    finally:
        conn.close()

    return text

Let's first check that functions work on its own, without agent

In [13]:
tables = db_query("SELECT name FROM sqlite_master WHERE type='table';")
print("Tables:", tables)


Tables: name
lifespan_change
gene_criteria
gene_hallmarks
longevity_associations



## Making a naive sql agent

Now we will create an agent, It will handle simple tasks.

First, let's install the just-agent library with pip and set up API keys for Groq and OpenAI

In [6]:
!pip install just-agents-core==0.7.3
!pip install just-agents-tools==0.7.3


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
  Attempting uninstall: just-agents-examples
    Found existing installation: just-agents-examples 0.5.9.post11+1de4a9d
    Uninstalling just-agents-examples-0.5.9.post11+1de4a9d:
      Successfully uninstalled just-agents-examples-0.5.9.post11+1de4a9d

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [14]:
import os
import getpass
from dotenv import load_dotenv

def get_api_keys():
  """Gets API keys from the user."""
  load_dotenv(override=True)

  # Check if GROQ_API_KEY is already set in environment
  groq_api_key = os.environ.get("GROQ_API_KEY")
  openai_api_key = os.environ.get("OPENAI_API_KEY")

  # Only prompt for key if not already set
  if not groq_api_key:
    print("Please enter your API keys:")
    groq_api_key = getpass.getpass("GROQ_API_KEY: ")
    # Set environment variable
    os.environ["GROQ_API_KEY"] = groq_api_key
    print("GROQ_API key set successfully.")
  else:
    print("GROQ API key already set in environment.")

  if not openai_api_key:
      print("Please enter your API keys:")
      openai_api_key = getpass.getpass("OPENAI_API_KEY: ")
      # Set environment variable
      os.environ["OPENAI_API_KEY"] = openai_api_key
      print("OPENAI_API key set successfully.")
  else:
    print("OPENAI API key already set in environment.")

# Call the function to get API keys from the user
get_api_keys()

GROQ API key already set in environment.


The following code just allows for user to see all comands and text outputed in the interface- it is needed to see all that comes out if the output is large.

In [15]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


Then we will create a simple agent that will use the db_query function as a tool and will answer questions which involve the database

Let's ask the agent some simple questions

In [16]:
from just_agents import llm_options
from just_agents.base_agent import BaseAgent




prompt = "What processes are improved in GHR knockout mice?"

session: BaseAgent = BaseAgent(
    llm_options=llm_options.OPENAI_GPT4oMINI,
    tools=[db_query]
)


result = session.query(prompt)

session.memory.pretty_print_all_messages()

╭─────────────────────────────────────────────────── [System] ────────────────────────────────────────────────────╮
│ You are a helpful AI assistant                                                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────── [User] ─────────────────────────────────────────────────────╮
│ What processes are improved in GHR knockout mice?                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── [Assistant] ──────────────────────────────────────────────────╮
│ Tool Call to db_query:                                                                                          │
│ {"sql":"SELECT processes_improved FROM ghr_knockout_mice_research"}                                             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────── [Tool] ─────────────────────────────────────────────────────╮
│ Response from db_query:                                                                                         │
│ no such table: ghr_knockout_mice_research                                                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── [Assistant] ──────────────────────────────────────────────────╮
│ It seems that I don't have access to a specific database regarding GHR knockout mice research. However, I can   │
│ provide general information based on existing knowledge.                                                        │
│                                                                                                                 │
│ GHR (Growth Hormone Receptor) knockout mice are often studied to understand the effects of growth hormone       │
│ signaling on various biological processes. Some of the processes that may be improved or altered in GHR         │
│ knockout mice include:                                                                                          │
│                                                                                                                 │
│ 1. **Longevity**: GHR knockout mice tend to have an increased lifespan compared to their wild-type              │
│ counterparts.                                                                                                   │
│ 2. **Metabolism**: These mice often exhibit improved insulin sensitivity and altered lipid metabolism, which    │
│ can lead to a healthier metabolic profile.                                                                      │
│ 3. **Body Composition**: GHR knockout mice typically have reduced body fat and increased lean body mass.        │
│ 4. **Stress Resistance**: Some studies suggest that GHR knockout mice may have enhanced resistance to certain   │
│ stressors, including oxidative stress.                                                                          │
│ 5. **Reproductive Health**: There may be changes in reproductive function and fertility, although the effects   │
│ can vary.                                                                                                       │
│                                                                                                                 │
│ If you need more specific information or studies, please let me know!                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────── [User] ─────────────────────────────────────────────────────╮
│ What processes are improved in GHR knockout mice?                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── [Assistant] ──────────────────────────────────────────────────╮
│ GHR (Growth Hormone Receptor) knockout mice have been studied extensively, and several processes have been      │
│ found to be improved or altered in these mice. Here are some key areas:                                         │
│                                                                                                                 │
│ 1. **Longevity**: GHR knockout mice often exhibit increased lifespan compared to wild-type mice. This is        │
│ thought to be related to reduced growth hormone signaling, which is associated with aging.                      │
│                                                                                                                 │
│ 2. **Metabolic Health**: These mice typically show improved insulin sensitivity and glucose metabolism, leading │
│ to a lower risk of developing metabolic disorders such as obesity and type 2 diabetes.                          │
│                                                                                                                 │
│ 3. **Body Composition**: GHR knockout mice generally have a lower body fat percentage and increased lean muscle │
│ mass. This change in body composition is linked to altered energy expenditure and metabolism.                   │
│                                                                                                                 │
│ 4. **Stress Resistance**: GHR knockout mice may demonstrate enhanced resistance to oxidative stress and other   │
│ forms of cellular stress, which can contribute to their longevity and overall health.                           │
│                                                                                                                 │
│ 5. **Cardiovascular Health**: Some studies suggest that GHR knockout mice have improved cardiovascular function │
│ and reduced incidence of age-related cardiovascular diseases.                                                   │
│                                                                                                                 │
│ 6. **Reproductive Function**: There can be alterations in reproductive health, including changes in fertility   │
│ and reproductive hormone levels, although the effects can vary.                                                 │
│                                                                                                                 │
│ 7. **Bone Density**: GHR knockout mice may exhibit changes in bone density and structure, often showing         │
│ increased bone mass due to reduced growth hormone signaling.                                                    │
│                                                                                                                 │
│ These improvements highlight the complex role of growth hormone signaling in various physiological processes    │
│ and its impact on health and aging. If you have specific studies or aspects you would like to explore further,  │
│ please let me know!                                                                                             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

It got this question right, but what if we want to know "Interventions on which genes extended mice lifespan most of all?"

In [19]:
from just_agents.base_agent import BaseAgent
from just_agents import llm_options


prompt = "Interventions on which genes extended mice lifespan most of all?"

agent: BaseAgent = BaseAgent(
    llm_options=llm_options.OPENAI_GPT4oMINI,
    tools=[db_query]
)


result = agent.query(prompt)
agent.memory.pretty_print_all_messages()

╭─────────────────────────────────────────────────── [System] ────────────────────────────────────────────────────╮
│ You are a helpful AI assistant                                                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────── [User] ─────────────────────────────────────────────────────╮
│ Interventions on which genes extended mice lifespan most of all?                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── [Assistant] ──────────────────────────────────────────────────╮
│ Tool Call to db_query:                                                                                          │
│ {"sql":"SELECT gene_symbol, max_lifespan_increase FROM genes ORDER BY max_lifespan_increase DESC LIMIT 5;"}     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────── [Tool] ─────────────────────────────────────────────────────╮
│ Response from db_query:                                                                                         │
│ no such table: genes                                                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── [Assistant] ──────────────────────────────────────────────────╮
│ Tool Call to db_query:                                                                                          │
│ {"sql":"SELECT gene_symbol, max_lifespan_increase FROM open_genes ORDER BY max_lifespan_increase DESC LIMIT     │
│ 5;"}                                                                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────── [Tool] ─────────────────────────────────────────────────────╮
│ Response from db_query:                                                                                         │
│ no such table: open_genes                                                                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── [Assistant] ──────────────────────────────────────────────────╮
│ It appears there is a persistent issue with accessing the gene database. Unfortunately, I cannot retrieve the   │
│ information about gene interventions that extended mice lifespan the most at this moment. If you have any other │
│ questions or need assistance with something else, feel free to ask!                                             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

As you see the SQL returned zero results although we know that the data exist by browsing https://open-genes.com/ and/or sqlite file. Let's modify the system prompt to get it right

In this case it was the simple change from model OPENAI-GPT4oMINI to OPENAI-GPT4o.

But how to make it better? simply call the LLMSession with a prompt.

In [22]:
from just_agents.base_agent import BaseAgent
from just_agents import llm_options

prompt = "Interventions on which genes extended mice lifespan most of all? Search all the relevant tables in the open-genes sqlite and only for mouse"

agent: BaseAgent = BaseAgent(
    llm_options=llm_options.OPENAI_GPT4o,
    system_prompt='open_genes.txt',
    tools=[db_query]
)


result = agent.query(prompt)
agent.memory.pretty_print_all_messages()

RESULT+++++++++++++++++++++++++++++++++++++++++++++++
The interventions on genes that extended the lifespan of mice the most are as follows:

1. **PROP1**: Improved insulin sensitivity and INS/IGFR pathway, resulting in a 60% lifespan extension.
2. **PROP1**: No specific improvement noted, resulting in a 46% lifespan extension.
3. **PIK3CA**: No specific improvement noted, resulting in a 45% lifespan extension.
4. **PROP1**: Improved glucose metabolism and INS/IGFR pathway, resulting in a 44.7% lifespan extension.
5. **PROP1**: No specific improvement noted, resulting in a 44% lifespan extension.

The gene **PROP1** appears multiple times with significant lifespan extensions under different conditions.


╭─────────────────────────────────────────────────── [System] ────────────────────────────────────────────────────╮
│ open_genes.txt                                                                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────── [User] ─────────────────────────────────────────────────────╮
│ Interventions on which genes extended mice lifespan most of all? Search all the relevant tables in the          │
│ open-genes sqlite and only for mouse                                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── [Assistant] ──────────────────────────────────────────────────╮
│ Tool Call to db_query:                                                                                          │
│ {"sql":"SELECT gene_symbol, intervention, lifespan_extension FROM lifespan_interventions WHERE species =        │
│ 'mouse' ORDER BY lifespan_extension DESC LIMIT 5;"}                                                             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────── [Tool] ─────────────────────────────────────────────────────╮
│ Response from db_query:                                                                                         │
│ no such table: lifespan_interventions                                                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭──────────────────────────────────────────────────── [Tool] ─────────────────────────────────────────────────────╮
│ Response from db_query:                                                                                         │
│ cid; name; type; notnull; dflt_value; pk                                                                        │
│ 0; HGNC; TEXT; 0; None; 0                                                                                       │
│ 1; hallmarks of aging; TEXT; 0; None; 0                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────── [Tool] ─────────────────────────────────────────────────────╮
│ Response from db_query:                                                                                         │
│ cid; name; type; notnull; dflt_value; pk                                                                        │
│ 0; HGNC; TEXT; 0; None; 0                                                                                       │
│ 1; polymorphism type; TEXT; 0; None; 0                                                                          │
│ 2; polymorphism id; TEXT; 0; None; 0                                                                            │
│ 3; nucleotide substitution; TEXT; 0; None; 0                                                                    │
│ 4; amino acid substitution; TEXT; 0; None; 0                                                                    │
│ 5; polymorphism — other; TEXT; 0; None; 0                                                                       │
│ 6; ethnicity; TEXT; 0; None; 0                                                                                  │
│ 7; study type; TEXT; 0; None; 0                                                                                 │
│ 8; sex; TEXT; 0; None; 0                                                                                        │
│ 9; doi; TEXT; 0; None; 0                                                                                        │
│ 10; pmid; TEXT; 0; None; 0                                                                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── [Assistant] ──────────────────────────────────────────────────╮
│ Tool Call to db_query:                                                                                          │
│ {"sql":"SELECT HGNC, intervention_improves, lifespan_percent_change_max FROM lifespan_change WHERE              │
│ model_organism = 'mouse' ORDER BY lifespan_percent_change_max DESC LIMIT 5;"}                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────── [Tool] ─────────────────────────────────────────────────────╮
│ Response from db_query:                                                                                         │
│ HGNC; intervention_improves; lifespan_percent_change_max                                                        │
│ PROP1; insulin sensitivity,INS/IGFR pathway; 60.0                                                               │
│ PROP1; None; 46.0                                                                                               │
│ PIK3CA; None; 45.0                                                                                              │
│ PROP1; glucose metabolism,INS/IGFR pathway; 44.7                                                                │
│ PROP1; None; 44.0                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── [Assistant] ──────────────────────────────────────────────────╮
│ The interventions on genes that extended the lifespan of mice the most are as follows:                          │
│                                                                                                                 │
│ 1. **PROP1**: Improved insulin sensitivity and INS/IGFR pathway, resulting in a 60% lifespan extension.         │
│ 2. **PROP1**: No specific improvement noted, resulting in a 46% lifespan extension.                             │
│ 3. **PIK3CA**: No specific improvement noted, resulting in a 45% lifespan extension.                            │
│ 4. **PROP1**: Improved glucose metabolism and INS/IGFR pathway, resulting in a 44.7% lifespan extension.        │
│ 5. **PROP1**: No specific improvement noted, resulting in a 44% lifespan extension.                             │
│                                                                                                                 │
│ The gene **PROP1** appears multiple times with significant lifespan extensions under different conditions.      │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Now to check it let's call the same code but with model OPENAIgpt4omini

In [ ]:
from just_agents.base_agent import BaseAgent
from just_agents import llm_options

prompt = "Interventions on which genes extended mice lifespan most of all? Search all the relevant tables in the open-genes sqlite and only for mouse"


session: BaseAgent = BaseAgent(
    llm_options=llm_options.OPENAI_GPT4oMINI,
    system_prompt='open_genes.txt',
    tools=[db_query]
)

result = agent.query(prompt)
agent.memory.pretty_print_all_messages()


{'content': 'Interventions on which genes extended mice lifespan most of all? '
            'Search all the relevant tables in the open-genes sqlite and only '
            'for mouse',
 'role': 'user'}
{'role': 'assistant',
 'tool_calls': [{'function': {'arguments': '{"sql": "SELECT HGNC, '
                                           'effect_on_lifespan, '
                                           'control_lifespan_mean, '
                                           'experiment_lifespan_mean, '
                                           'lifespan_percent_change_mean FROM '
                                           'lifespan_change WHERE '
                                           "model_organism = 'mouse' AND "
                                           'effect_on_lifespan LIKE '
                                           "'%increases lifespan%' ORDER BY "
                                           'lifespan_percent_change_mean DESC '
                                           'LIMIT 

This shows that even with careful arragement prompts can still fail. It is important to check the results and stability of results before anything else.

Here are some more questions to be tested

1. What genes need to be downregulated in worms to extend their lifespan?
2. What processes are improved in GHR knockout mice?
3. Which genetic intervention led to the greatest increase in lifespan in flies?
4. To what extent did the lifespan increase in mice overexpressing VEGFA?
5. Are there any liver-specific interventions that increase lifespan in mice?
6. Which gene-longevity association is confirmed by the greatest number of studies?
7. What polymorphisms in FOXO3 are associated with human longevity?
8. In which ethnic groups was the association of the APOE gene with longevity shown?
9. Is the INS gene polymorphism associated with longevity?
10. What genes are associated with transcriptional alterations?
11. Which hallmarks are associated with the KL gene?
12. What genes change their expression with aging in humans?
13. How many genes are associated with longevity in humans?
14. What types of studies have been conducted on the IGF1R gene?
15. What evidence of the link between PTEN and aging do you know?
16. What genes are associated with both longevity and altered expression in aged humans?
17. Is the expression of the ACE2 gene altered with aging in humans?
18. Interventions on which genes extended mice lifespan most of all?
19. Which knockdowns were most lifespan extending on model animals?